In [1]:
import numpy as np
import tensorflow as tf

import tensorflow_datasets as tfds

In [2]:
mnist_dataset, mnist_info = tfds.load(name='mnist', with_info=True, as_supervised=True)

In [3]:
mnist_train, mnist_test = mnist_dataset['train'], mnist_dataset['test']

In [4]:
num_validation_samples = 0.1 * mnist_info.splits['train'].num_examples
num_validation_samples = tf.cast(num_validation_samples, tf.int64)

In [5]:
num_test_samples = mnist_info.splits['test'].num_examples
num_test_samples = tf.cast(num_test_samples, tf.int64)

In [6]:
def scale(image, label):
    image = tf.cast(image, tf.float32)
    image /= 255.
    return image, label

In [7]:
scaled_train_and_validation_data = mnist_train.map(scale)
test_data = mnist_test.map(scale)

In [8]:
BUFFER_SIZE = 10000
shuffled_train_and_validation_data = scaled_train_and_validation_data.shuffle(BUFFER_SIZE)
validation_data = shuffled_train_and_validation_data.take(num_validation_samples)
train_data = shuffled_train_and_validation_data.skip(num_validation_samples)

In [9]:
BATCH_SIZE = 100
train_data = train_data.batch(BATCH_SIZE)
validation_data = validation_data.batch(num_validation_samples)
test_data = test_data.batch(num_test_samples)
validation_inputs, validation_targets = next(iter(validation_data))

In [10]:
input_size = 784
# We don't need to define input size in TF2.0 
output_size = 10
hidden_layer_size = 300
    
model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28, 1)),
    tf.keras.layers.Dense(hidden_layer_size, activation='relu'), # input layer
    tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
    tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
    tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
    
    tf.keras.layers.Dense(output_size, activation='softmax') # output layer
])

In [11]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [12]:
NUM_EPOCHS = 5
early_stopping = tf.keras.callbacks.EarlyStopping(patience=1)
VALIDATION_STEPS = num_validation_samples

model.fit(train_data, epochs=NUM_EPOCHS, callbacks=[early_stopping], validation_data=(validation_inputs, validation_targets), validation_steps=VALIDATION_STEPS, verbose =2)

Epoch 1/5
540/540 - 50s - loss: 0.2434 - accuracy: 0.9268 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 2/5
540/540 - 51s - loss: 0.0979 - accuracy: 0.9695 - val_loss: 0.0872 - val_accuracy: 0.9717
Epoch 3/5
540/540 - 53s - loss: 0.0706 - accuracy: 0.9779 - val_loss: 0.0714 - val_accuracy: 0.9783
Epoch 4/5
540/540 - 54s - loss: 0.0534 - accuracy: 0.9832 - val_loss: 0.0555 - val_accuracy: 0.9830
Epoch 5/5
540/540 - 56s - loss: 0.0455 - accuracy: 0.9860 - val_loss: 0.0427 - val_accuracy: 0.9857


In [13]:
test_loss, test_accuracy = model.evaluate(test_data)

1/1 [==============================] - 3s 3s/step - loss: 0.0730 - accuracy: 0.9783


In [14]:
print('Test loss: {0:.2f}. Test accuracy: {1:.2f}%'.format(test_loss, test_accuracy*100.))

Test loss: 0.07. Test accuracy: 97.83%
